In [ ]:
import pandas as pd
import psycopg2
from dotenv import load_dotenv
import json
import os
from tabulate import tabulate

# Umgebungsvariablen laden
load_dotenv()

# Load DDC categories
with open("../data/ddc/ddc-basic.json", "r", encoding="utf-8") as f:
    ddc_categories = json.load(f)

# Create a mapping of DDC numbers to names
ddc_names = {str(k): v["name"] for k, v in ddc_categories.items()}


def connect_to_db() -> psycopg2.extensions.connection:
    """Datenbankverbindung herstellen"""
    return psycopg2.connect(
        host=os.getenv("DB_HOST"),
        port=os.getenv("DB_PORT"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
    )


# Connect to database
conn = connect_to_db()

# Query to get the data with year cast to integer
query = """
SELECT 
    SUBSTRING(ddc FROM 1 FOR 1) as ddc_category,
    CAST(year AS INTEGER) as year,
    COUNT(*) as count
FROM dnb_records_subset
WHERE year IS NOT NULL
  AND year ~ '^[0-9]+$'  -- ensure year contains only digits
GROUP BY SUBSTRING(ddc FROM 1 FOR 1), year
ORDER BY year, ddc_category;
"""

# Read data into DataFrame
df = pd.read_sql_query(query, conn)

# Define year distribution categories
year_bins = [-float("inf"), 2000, 2010, 2015, 2020, 2024]
year_labels = ["<2000", "2000-2010", "2010-2015", "2015-2020", "2020-2024"]

# Add a new column for year distribution
df["year_distribution"] = pd.cut(df["year"], bins=year_bins, labels=year_labels, right=False)

# Calculate total count per DDC category
ddc_total_count = df.groupby("ddc_category")["count"].sum()

# Calculate total entries for percentage calculation
total_entries = ddc_total_count.sum()

# Print total entries
print(f"\nTotal number of entries: {total_entries}")

# Create a DataFrame for total counts with percentage
total_counts_df = pd.DataFrame(
    {
        "DDC": [f"{ddc}" for ddc in ddc_total_count.index],
        "Category": [ddc_names.get(ddc, "Unknown") for ddc in ddc_total_count.index],
        "Count": ddc_total_count.values,
        "Percentage": ((ddc_total_count.values / total_entries) * 100).round(1).astype(str) + "%",
    }
).sort_values(
    "DDC"
)  # Sort by DDC number

# Add total row to total_counts_df
total_row = pd.DataFrame([{"DDC": "Total", "Category": "", "Count": total_entries, "Percentage": "100.0%"}])
total_counts_df = pd.concat([total_counts_df, total_row])

print("\nTotal count per DDC category:")
print(
    tabulate(
        total_counts_df,
        headers="keys",
        tablefmt="pretty",
        showindex=False,
        colalign=("right", "left", "right", "right"),
    )
)

# Calculate count per year distribution per DDC category
ddc_year_distribution = df.groupby(["ddc_category", "year_distribution"])["count"].sum().unstack(fill_value=0)

# Filter out unknown category
ddc_year_distribution = ddc_year_distribution[ddc_year_distribution.index.str.match(r"[0-9]")]

# Split DDC and Category into separate columns
ddc_year_distribution = ddc_year_distribution.reset_index()
ddc_year_distribution["Category"] = ddc_year_distribution["ddc_category"].map(ddc_names)

# Reorder columns to have DDC and Category first
ddc_year_distribution = ddc_year_distribution[["ddc_category", "Category"] + list(ddc_year_distribution.columns[1:-1])]

# Calculate totals for year distribution
year_totals = ddc_year_distribution.iloc[:, 2:].sum()
total_row = pd.DataFrame([{"ddc_category": "Total", "Category": "", **year_totals.to_dict()}])
ddc_year_distribution = pd.concat([ddc_year_distribution, total_row])

print("\nCount per year distribution per DDC category:")
print(
    tabulate(
        ddc_year_distribution,
        headers=["DDC", "Category"] + list(ddc_year_distribution.columns[2:]),
        tablefmt="pretty",
        showindex=False,
        colalign=("right", "left", "right", "right", "right", "right", "right"),
    )
)

# Query to get token statistics
token_query = """
WITH token_ranges AS (
    SELECT 
        COUNT(*) as count,
        CASE 
            WHEN token_count < 50000 THEN '<50k'
            WHEN token_count >= 50000 AND token_count < 80000 THEN '50-80k'
            WHEN token_count >= 80000 AND token_count < 100000 THEN '80-100k'
            WHEN token_count >= 100000 AND token_count < 128000 THEN '100-128k'
            ELSE '>128k'
        END as token_range
    FROM dnb_records_subset
    WHERE token_count IS NOT NULL
    GROUP BY 
        CASE 
            WHEN token_count < 50000 THEN '<50k'
            WHEN token_count >= 50000 AND token_count < 80000 THEN '50-80k'
            WHEN token_count >= 80000 AND token_count < 100000 THEN '80-100k'
            WHEN token_count >= 100000 AND token_count < 128000 THEN '100-128k'
            ELSE '>128k'
        END
)
SELECT * FROM token_ranges
ORDER BY 
    CASE token_range
        WHEN '<50k' THEN 1
        WHEN '50-80k' THEN 2
        WHEN '80-100k' THEN 3
        WHEN '100-128k' THEN 4
        WHEN '>128k' THEN 5
    END;
"""

# Read token data into DataFrame and reorder columns
token_df = pd.read_sql_query(token_query, conn)
token_df = token_df[["token_range", "count"]]  # Explicitly set column order

# Calculate percentages
total_tokens = token_df["count"].sum()
token_df["percentage"] = ((token_df["count"] / total_tokens) * 100).round(1).astype(str) + "%"

# Add total row
total_row = pd.DataFrame([{"token_range": "Total", "count": total_tokens, "percentage": "100.0%"}])
token_df = pd.concat([token_df, total_row])

# Print token distribution table
print("\nToken count distribution:")
print(
    tabulate(
        token_df,
        headers=["Range", "Count", "Percentage"],
        tablefmt="pretty",
        showindex=False,
        colalign=("left", "right", "right"),
    )
)

# Close connection
conn.close()